# Предобработка датасета

Подключаем диск

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report, mean_squared_error
from sklearn.impute import KNNImputer

Считываем датасет

In [ ]:
result_slice=pd.read_csv('/content/drive/MyDrive/result_slice (3).csv')

In [ ]:
result_slice = result_slice.dropna(axis='columns', how='all', inplace=False)
result_slice = result_slice.drop('Unnamed: 0', axis=1)

result_slice_nan = pd.DataFrame(round((result_slice.isnull().mean() * 100), 2))
result_slice = result_slice.drop(list(result_slice_nan[result_slice_nan[0] > 40].index), axis=1)

#Сформируем столбец таргета
result_slice= result_slice.reindex(columns=list(result_slice.drop('EF_b', axis=1).append(pd.DataFrame(result_slice['EF_b']))))
result_slice['EF_b_target'] = (result_slice['EF_b'] > 2).astype('float64')

In [ ]:
result_slice=result_slice.dropna()

# Нейросеть для классификации

Таргет и предикторы

In [ ]:
X = result_slice.drop(['DEPT', 'well', 'FZ', 'EF_b', 'EF_b_target'], axis=1)
y = result_slice['EF_b_target']

In [ ]:
from sklearn.preprocessing import StandardScaler
stdScaler = StandardScaler()
X = stdScaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_all_train, X_test, y_all_train, y_test = train_test_split(X, y, test_size=0.15, random_state=15)

X_train, X_val, y_train, y_val = train_test_split(X_all_train, y_all_train, test_size=0.2,random_state=15)

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
batch_size = 20
max_iter = 1000

In [ ]:
model = MLPClassifier(hidden_layer_sizes=(1,2), activation='tanh', early_stopping=True, max_iter=max_iter, batch_size=batch_size)
hist_cl = model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

         0.0       0.85      0.90      0.87       580
         1.0       0.92      0.88      0.90       742

    accuracy                           0.89      1322
   macro avg       0.88      0.89      0.89      1322
weighted avg       0.89      0.89      0.89      1322

